In [1]:
!pip install openai==0.28

In [30]:
import sqlite3
conn.close()

conn = sqlite3.connect('sample.db')
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS employees')

cursor.execute('''
    CREATE TABLE employees (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        age INTEGER,
        department TEXT
    )
''')

cursor.execute('INSERT INTO employees (name, age, department) VALUES (?, ?, ?)', ("Alice", 30, "HR"))
cursor.execute('INSERT INTO employees (name, age, department) VALUES (?, ?, ?)', ("Bob", 24, "Engineering"))
cursor.execute('INSERT INTO employees (name, age, department) VALUES (?, ?, ?)', ("Charlie", 29, "Sales"))

conn.commit()


In [31]:
import openai

openai.api_key = 'sk-proj-BkpeMAvYI6HACybicGBmT3BlbkFJZQaQ08lphOKJFnysDLIG'

In [32]:
def get_sql_query(natural_language_query):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Convert th  e following natural language query to an SQL query:\n\n{natural_language_query}"}
        ]
    )

    response_text = response.choices[0].message['content'].strip()

    start_idx = response_text.find("```sql")
    end_idx = response_text.find("```", start_idx + 1)


    if start_idx != -1 and end_idx != -1:
        sql_query = response_text[start_idx + 6:end_idx].strip()
    else:
        sql_query = response_text

    return sql_query


In [33]:
def execute_sql_query(sql_query):
    cursor.execute(sql_query)
    rows = cursor.fetchall()
    return rows

def display_results(rows):
    for row in rows:
        print(row)


In [37]:

nl_query = "Show the person's name who is working in the Engineering dept"

sql_query = get_sql_query(nl_query)
print(f"Generated SQL Query: {sql_query}")

result = execute_sql_query(sql_query)

display_results(result)


Generated SQL Query: SELECT name
FROM employees
WHERE department = 'Engineering';
('Bob',)
